# 6. Дополнительные признаки

На этой неделе вам предстоит попробовать добавить в вашу регрессионную модель дополнительные признаки.

Во-первых, для прогнозирования можно использовать информацию, содержащуюся в сырых данных:

- средняя длительность поездок
- среднее количество пассажиров
- среднее расстояние по счётчику
- доли географических зон, в которые совершаются поездки
- доли поездок, совершаемых по тарифам каждого из типов
- доли способов оплаты поездок
- средняя стоимость поездок
- доли провайдеров данных

Все эти признаки можно использовать только с задержкой, то есть, при прогнозировании $\hat{y}_{T+i|T}$ эти признаки должны быть рассчитаны по данным не позднее момента времени $T$. Каждый из этих признаков можно использовать по-разному: как сырые значения за последние несколько часов, так и средние за последний день, неделю, месяц и т. д.

Во-вторых, чтобы улучшить качество прогнозов в аномальные периоды, вы можете найти информацию о потенциально влияющих на количество поездок событиях, таких, как государственные праздники. Проанализируйте, как именно поведение пассажиров меняется во время этих событий, и создайте признаки, отражающие эти изменения. Как показывает наш опыт, правильный учёт праздничных дней часто позволяет существенно уменьшить среднюю ошибку прогноза.

В-третьих, можно использовать признаки, связанные с географией. Например, скорее всего, суммарное количество поездок, совершаемых из географической зоны, пропорционально площади этой зоны. Для зон, прилегающих к аэропорту, может быть характерен специфический паттерн дневной сезонности, связанный с тем, что спрос на такси будет повышаться в те часы, когда общественный транспорт перестаёт работать. В деловом центре максимальное количество поездок будет приходиться на начало и окончание рабочего дня, на Бродвее — на время начала и окончания спектаклей. Все эти идеи не обязательно верны, мы приводим их здесь только для того, чтобы продемонстрировать принцип рассуждений. Ещё один пример географического признака: можно попробовать добавить идентификатор боро, который можно найти в файле https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv. Кроме того, нам кажется перспективным использование в качестве фактора количества поездок, совершённых за прошлый час/день и т. д. из соседних географических зон, или количества поездок, совершённых за прошлый час/день в текущую географическую зону.

Много примеров других признаков, которые можно использовать при регрессионном прогнозировании, можно найти в [лекции](https://habrahabr.ru/company/yandex/blog/316232) Вадима Стрижова.



## Задание № 1

Попробуйте добавить признаки. Используйте идеи, которые мы предложили, или какие-то свои. Обучайте обновлённые модели на данных до апреля 2016 включительно и считайте качество новых прогнозов на мае. Удаётся ли вам улучшить качество? Не нужно ли увеличить сложность регрессионной модели? Если добавляемый признак не улучшает качество, всё равно оставьте доказательства этому в ноутбуке, чтобы ваши коллеги это видели при проверке.

In [1]:
import numpy as np
import pandas as pd
from glob import glob

In [2]:
# Функция разделения единого массива данных на обучающую, тестовую и контрольную выборки

def split_data(input_data):
    train_data = pd.concat([input_data[input_data.year < 2016],
                            input_data[(input_data.year == 2016) & (input_data.month.astype(int) < 5)]])

    test_data = input_data[(input_data.year == 2016) & (input_data.month == "5")]
    control_data = pd.concat([test_data[-102:],
                             input_data[(input_data.year == 2016) & (input_data.month == "6")]])
    test_data = test_data[:-102]
    
    return train_data, test_data, control_data

In [3]:
file_path = np.sort(glob("new_data/*.csv", recursive=True))
input_data = []
for path in file_path:
    new_data = pd.read_csv(path)
    new_data.month = new_data.month.astype(str)
    input_data.append(new_data)
    
result_data = []
for i in range(len(input_data)):
    i_data = input_data[i]
    result_data.append(split_data(i_data))

In [4]:
# Функция считывания оргинальных данных
def read_data(name, state):
    columns = []
    if state == "bad":
        columns = ["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count",
               "trip_distance", "RatecodeID", "store_and_fwd_flag", "PULocationID", "DOLocationID",
               "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount",
               "improvement_surcharge", "total_amount", "", ""]
    else:
        columns = ["vendor_id", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count",
               "trip_distance", "pickup_longitude", 'pickup_latitude', "rate_code", "store_and_fwd_flag",
               "dropoff_longitude", "dropoff_latitude", "payment_type", "fare_amount",
               "surcharge", "mta_tax", 'tip_amount', "tolls_amount", "total_amount"]
        
    parse_dates = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]
    data = {
    'normal': lambda name: pd.read_csv("origin_data/" + name, header=0,
                   sep=",", parse_dates=parse_dates),
    'bad': lambda name: pd.read_csv("origin_data/" + name, sep=",", names=columns,
                   skiprows=1, parse_dates=parse_dates),
    'old': lambda name: pd.read_csv("origin_data/" + name, skiprows=1,
                   sep=",", parse_dates=parse_dates, names=columns)
    }[state](name)
    
    return data

In [3]:
from datetime import timedelta, datetime

# Функция построения массива дат на заданном интервале

def daterange(date1, date2, move=5):
    dates = []
    for n in range(int((date2 - date1).days * 24) + 24 - move):
        dates.append(date1 + timedelta(n / 24))
    return dates

In [6]:
most_regions = pd.unique(result_data[0][2]["region"])

# Функция считывания оргинальных данных с предварительным аггрегированием

def open_data(file_path):
    data = read_data(file_path, "normal")

    data = data[data.tpep_pickup_datetime != data.tpep_dropoff_datetime]
    data = data[data.passenger_count != 0]
    data = data[data.trip_distance != 0.0]
    
    region_data = pd.read_csv("regions.csv", sep=";")
    
    regions = []
    for region in region_data.get_values():
        region_id = int(region[0])
        index = data[(data["pickup_longitude"].between(region[1], region[2])) & (data["pickup_latitude"].between(region[3], region[4]))].index
        region_vals = np.repeat("region_" + str(region_id), index.shape[0])
        regions.append(pd.DataFrame(region_vals, columns=["region"], index=index))
    
    regions = pd.concat(regions).sort_index()
    regions = regions[regions.index.duplicated() == False]
    data = data[data.index.isin(regions.sort_index().index)]
    
    columns = ["VendorID", "tpep_pickup_datetime", "passenger_count", "trip_distance",
               "store_and_fwd_flag", "payment_type", "total_amount"]
    
    data = pd.concat([data[columns], regions], axis=1) 
    data = data[data.region.isin(most_regions)]
    
    data.tpep_pickup_datetime = list(map(lambda x: x.replace(minute=0, second=0), data.tpep_pickup_datetime))
    data.store_and_fwd_flag = data.store_and_fwd_flag.fillna("UN")
    
    return data

In [7]:
# Функция преобразования признаков

def get_aggregate_region(data, region):
    dates = daterange(data.tpep_pickup_datetime.min(), data.tpep_pickup_datetime.max(), 0)
    res = pd.DataFrame({"region" : np.repeat(region, len(dates)), "time" : dates})
    
    cmt_res = data[(data.VendorID == 1) & (data.region == region)].groupby("tpep_pickup_datetime").count()
    cmt_res = pd.DataFrame({"time" : cmt_res.index.get_values(),
                            "CMT_vendor" : cmt_res["VendorID"].get_values()})

    res = pd.merge(res, cmt_res, how='left', on="time").fillna(0)
    
    vts_res = data[(data.VendorID == 2) & (data.region == region)].groupby("tpep_pickup_datetime").count()
    vts_res = pd.DataFrame({"time" : vts_res.index.get_values(),
                            "VTS_vendor" : vts_res["VendorID"].get_values()})
    
    res = pd.merge(res, vts_res, how='left', on="time").fillna(0)
    
    mean_res = data[data.region == region].groupby("tpep_pickup_datetime").mean()
    mean_res = pd.DataFrame({"time" : mean_res.index.get_values(),
                             "passenger_count" : mean_res["passenger_count"].get_values(),
                             "trip_distance" : mean_res["trip_distance"].get_values(),
                             "total_amount" : mean_res["total_amount"].get_values()})
    
    res = pd.merge(res, mean_res, how='left', on="time").fillna(0)
    
    csh_res = data[(data.payment_type == 2) & (data.region == region)].groupby("tpep_pickup_datetime").count()
    csh_res = pd.DataFrame({"time" : csh_res.index.get_values(),
                            "CSH_payment_type" : csh_res["payment_type"].get_values()})
    
    res = pd.merge(res, csh_res, how='left', on="time").fillna(0)
    
    crd_res = data[(data.payment_type == 1) & (data.region == region)].groupby("tpep_pickup_datetime").count()
    crd_res = pd.DataFrame({"time" : crd_res.index.get_values(),
                            "CRD_payment_type" : crd_res["payment_type"].get_values()})
    
    res = pd.merge(res, crd_res, how='left', on="time").fillna(0)
    
    dis_res = data[(data.payment_type == 4) & (data.region == region)].groupby("tpep_pickup_datetime").count()
    dis_res = pd.DataFrame({"time" : dis_res.index.get_values(),
                            "DIS_payment_type" : dis_res["payment_type"].get_values()})
    
    res = pd.merge(res, dis_res, how='left', on="time").fillna(0)
    
    noc_res = data[(data.payment_type == 3) & (data.region == region)].groupby("tpep_pickup_datetime").count()
    noc_res = pd.DataFrame({"time" : noc_res.index.get_values(),
                            "NOC_payment_type" : noc_res["payment_type"].get_values()})
    
    res = pd.merge(res, noc_res, how='left', on="time").fillna(0)
    
    unk_res = data[(data.payment_type == 5) & (data.region == region)].groupby("tpep_pickup_datetime").count()
    unk_res = pd.DataFrame({"time" : unk_res.index.get_values(),
                            "UNK_payment_type" : unk_res["payment_type"].get_values()})
    
    res = pd.merge(res, unk_res, how='left', on="time").fillna(0)
    
    n_res = data[(data.store_and_fwd_flag == "N") & (data.region == region)].groupby("tpep_pickup_datetime").count()
    n_res = pd.DataFrame({"time" : n_res.index.get_values(),
                          "N_store_and_fwd_flag" : n_res["store_and_fwd_flag"].get_values()})
    
    res = pd.merge(res, n_res, how='left', on="time").fillna(0)
    
    un_res = data[(data.store_and_fwd_flag == "UN") & (data.region == region)].groupby("tpep_pickup_datetime").count()
    un_res = pd.DataFrame({"time" : un_res.index.get_values(),
                           "UN_store_and_fwd_flag" : un_res["store_and_fwd_flag"].get_values()})
    
    res = pd.merge(res, un_res, how='left', on="time").fillna(0)
    
    y_res = data[(data.store_and_fwd_flag == "Y") & (data.region == region)].groupby("tpep_pickup_datetime").count()
    y_res = pd.DataFrame({"time" : y_res.index.get_values(),
                          "Y_store_and_fwd_flag" : y_res["store_and_fwd_flag"].get_values()})
    
    res = pd.merge(res, y_res, how='left', on="time").fillna(0)
    
    return res

In [8]:
import os

# Функция подготовки данных для слияния с исходной выборкой

def prepare_data(file_name):
    data = open_data(file_name)
    
    trans_res = []
    for region in most_regions:
        trans_res.append(get_aggregate_region(data, region))
    
    res_data = pd.concat(trans_res)
    res_data.to_csv("trans_data/" + file_name, index=False)
    
    os.remove("origin_data/" + file_name)
    
    del data
    del res_data

In [605]:
%%time

path_list = os.listdir("origin_data")

print("Begin!")
for file_name in path_list:
    print(file_name)
    prepare_data(file_name)
    
print("Done!")

Begin!
yellow_tripdata_2015-05.csv
yellow_tripdata_2015-11.csv
yellow_tripdata_2015-10.csv
yellow_tripdata_2015-04.csv
yellow_tripdata_2015-12.csv
yellow_tripdata_2015-06.csv
yellow_tripdata_2015-07.csv
yellow_tripdata_2015-03.csv
yellow_tripdata_2015-02.csv
yellow_tripdata_2015-01.csv
yellow_tripdata_2016-02.csv
yellow_tripdata_2016-03.csv
yellow_tripdata_2016-01.csv
yellow_tripdata_2016-04.csv
yellow_tripdata_2016-05.csv
yellow_tripdata_2016-06.csv
yellow_tripdata_2015-09.csv
yellow_tripdata_2015-08.csv
Done!
CPU times: user 8h 14min 58s, sys: 30min 11s, total: 8h 45min 10s
Wall time: 8h 47min 41s


На данном этапе происходит подготовка данных с сайта открытых данных Нью Йорка

In [217]:
columns = ["time", "visibility", "temp", "dew_point_temp",
           "relative_humidity", "wind_speed"]

weather_data = pd.read_csv("weather_data.csv", date_parser=["DATE"])

weather_data["time"] = list(map(lambda x: x.replace(minute=0, second=0), weather_data.DATE.astype(np.datetime64)))
weather_data["visibility"] = list(map(lambda x: float(str(x).split("V")[0]),weather_data.HOURLYVISIBILITY))
weather_data["temp"] = weather_data.HOURLYWETBULBTEMPC
weather_data["dew_point_temp"] = list(map(lambda x: float(str(x).split("s")[0]),weather_data.HOURLYDewPointTempC))
weather_data["relative_humidity"] = weather_data.HOURLYRelativeHumidity
weather_data["wind_speed"] = weather_data.HOURLYWindSpeed

weather_data = weather_data[weather_data.time.duplicated() == False]
weather_data = weather_data[columns]
weather_data = weather_data.fillna(0.0)
weather_data.head(5)

/Users/Ar4eBaT/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,11,14,15,20,23,25,40,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Ar4eBaT/anaconda/lib/python3.6/site-packages/pandas/core/internals.py:3462: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


,time,visibility,temp,dew_point_temp,relative_humidity,wind_speed
0,2013-01-01 00:00:00,10.0,0.8,-4.4,55.0,10.0
1,2013-01-01 01:00:00,10.0,0.8,-3.3,62.0,3.0
2,2013-01-01 02:00:00,10.0,1.3,-2.8,62.0,7.0
3,2013-01-01 03:00:00,10.0,1.3,-2.8,62.0,9.0
4,2013-01-01 04:00:00,10.0,1.3,-2.8,62.0,7.0


На данном этапе определяются все государственные праздники

In [186]:
holidays = [datetime(2013, 1, 1), datetime(2013, 1, 21), datetime(2013, 2, 18), datetime(2013, 5, 27),
            datetime(2013, 7, 4), datetime(2013, 9, 2), datetime(2013, 10, 14), datetime(2013, 11, 11),
           datetime(2013, 11, 28), datetime(2013, 12, 25), datetime(2014, 1, 1), datetime(2014, 1, 20),
            datetime(2014, 2, 17), datetime(2014, 5, 26), datetime(2014, 7, 4), datetime(2014, 9, 1),
            datetime(2014, 10, 13), datetime(2014, 11, 11), datetime(2014, 11, 27), datetime(2014, 12, 25),
           datetime(2015, 1, 1), datetime(2015, 1, 19), datetime(2015, 2, 16), datetime(2015, 5, 25),
            datetime(2015, 7, 3), datetime(2015, 9, 7), datetime(2015, 10, 12), datetime(2015, 11, 11),
            datetime(2015, 11, 26), datetime(2015, 12, 25), datetime(2016, 1, 1), datetime(2016, 1, 18),
            datetime(2016, 2, 15), datetime(2016, 5, 30)]
dates = []
for holiday in holidays:
    dates.extend(daterange(holiday, holiday, 0))
    
holidays_data = pd.DataFrame({"time":dates, "holiday": np.repeat(True, len(dates))})
holidays_data.head(5)

,holiday,time
0,True,2013-01-01 00:00:00
1,True,2013-01-01 01:00:00
2,True,2013-01-01 02:00:00
3,True,2013-01-01 03:00:00
4,True,2013-01-01 04:00:00


In [221]:
attr_data = pd.merge(weather_data, holidays_data, how="left").fillna(False)
attr_data.head(5)

,time,visibility,temp,dew_point_temp,relative_humidity,wind_speed,holiday
0,2013-01-01 00:00:00,10.0,0.8,-4.4,55.0,10.0,True
1,2013-01-01 01:00:00,10.0,0.8,-3.3,62.0,3.0,True
2,2013-01-01 02:00:00,10.0,1.3,-2.8,62.0,7.0,True
3,2013-01-01 03:00:00,10.0,1.3,-2.8,62.0,9.0,True
4,2013-01-01 04:00:00,10.0,1.3,-2.8,62.0,7.0,True


In [238]:
path_list = os.listdir("trans_data")

data_list = []
for path in path_list:
    data_list.append(pd.read_csv("trans_data/"+path, date_parser=["time"]))
    
trans_data = pd.concat(data_list)
trans_data.time = trans_data.time.astype(np.datetime64)

/Users/Ar4eBaT/anaconda/lib/python3.6/site-packages/pandas/core/internals.py:3462: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


In [354]:
complete_attr = pd.merge(trans_data, attr_data, how="left", on="time").sort_values("time")
complete_attr.head(5)

,region,time,CMT_vendor,VTS_vendor,passenger_count,total_amount,trip_distance,CSH_payment_type,CRD_payment_type,DIS_payment_type,...,UNK_payment_type,N_store_and_fwd_flag,UN_store_and_fwd_flag,Y_store_and_fwd_flag,visibility,temp,dew_point_temp,relative_humidity,wind_speed,holiday
1449384,region_1280,2013-03-01,70.0,80.0,1.713333,14.699533,3.047933,69.0,80.0,0.0,...,1.0,68.0,80.0,2.0,10.0,1.4,-1.7,70.0,7.0,False
1486584,region_1684,2013-03-01,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,1.4,-1.7,70.0,7.0,False
1444176,region_1234,2013-03-01,118.0,100.0,1.816514,13.047431,2.910780,103.0,112.0,0.0,...,0.0,117.0,100.0,1.0,10.0,1.4,-1.7,70.0,7.0,False
1476912,region_1437,2013-03-01,18.0,15.0,2.212121,14.225758,3.569697,19.0,14.0,0.0,...,0.0,18.0,15.0,0.0,10.0,1.4,-1.7,70.0,7.0,False
1491792,region_2119,2013-03-01,45.0,57.0,1.941176,49.096569,14.718725,46.0,55.0,0.0,...,0.0,45.0,57.0,0.0,10.0,1.4,-1.7,70.0,7.0,False


In [357]:
complete_attr.to_csv("add_attributes.csv", index=False)

## Задание № 2

Когда вы примете решение остановиться и перестать добавлять признаки, постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:

<center>$Q_{june}=\frac{1}{R∗739∗6}\sum_{r=1}^R\sum_{T=2016.05.01-00:00}^{T=2016.05.31-23:00}\sum_{i=1}^6\mid{\widehat{y}}^r_{T|T+i}−{\widehat{y}}^r_{T+i}\mid.$</center>

Убедитесь, что среднее качество прогнозов увеличилось.

In [4]:
file_path = np.sort(glob("complete_data/*.csv", recursive=True))
input_data = []
for path in file_path:
    new_data = pd.read_csv(path)
    new_data.month = new_data.month.astype(str)
    new_data.hour = new_data.hour.astype(str)
    input_data.append(new_data)
    
result_data = []
for i in range(len(input_data)):
    i_data = input_data[i]
    result_data.append(split_data(i_data))

In [5]:
attribute_cols = input_data[0].columns[(input_data[0].columns != "trips") & (input_data[0].columns != "year") & (input_data[0].columns != "day") & (input_data[0].columns != "UNK_payment_type")]
attribute_cols

Index(['Trend', 'cos_week_1', 'cos_week_2', 'cos_week_3', 'cos_week_4',
       'cos_week_5', 'cos_week_6', 'cos_year_1', 'cos_year_10', 'cos_year_11',
       ...
       'NOC_payment_type', 'N_store_and_fwd_flag', 'UN_store_and_fwd_flag',
       'Y_store_and_fwd_flag', 'visibility', 'temp', 'dew_point_temp',
       'relative_humidity', 'wind_speed', 'holiday'],
      dtype='object', length=108)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer as DV

In [7]:
# cat_attributes = ["region", "month", "dayofweek", "hour"]
cat_attributes = ["region", "month", "dayofweek"]
regions = pd.unique(result_data[0][0]["region"])
month = list(map(lambda x: "m_" + str(x), (pd.unique(result_data[0][0]["month"]))))
# hours = list(map(lambda x: "h_" + str(x), (pd.unique(result_data[0][0]["hour"]))))
dayofweek = pd.unique(result_data[0][0]["dayofweek"])
# cat_columns = np.append(np.append(np.append(regions, month), dayofweek), hours)
cat_columns = np.append(np.append(regions, month), dayofweek)

In [8]:
encoder = DV(sparse = False)
cat_fit_data = result_data[0][0][cat_attributes].T.to_dict().values()
encoder.fit(cat_fit_data)

DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)

In [9]:
a = list(attribute_cols)
a.remove("month")
a.remove('dayofweek')
a.remove('hour')
a.remove('region')

In [ ]:
%%time

results = []
for data in result_data:
    estimator = RandomForestRegressor()
    
    train_data = data[0].replace([np.inf, -np.inf], np.nan).fillna(0.0)
    
    encoded_data = encoder.transform(train_data[cat_attributes].T.to_dict().values())
    cat_att = pd.DataFrame(encoded_data, columns=cat_columns)
    cat_att.index = train_data[a].index

    X_train = pd.concat([train_data[a], cat_att], axis=1)
    y_train = train_data["trips"]

    fitted = estimator.fit(X_train, y_train)
    
    test_data = data[1].replace([np.inf, -np.inf], np.nan).fillna(0.0)
    
    encoded_data = encoder.transform(test_data[cat_attributes].T.to_dict().values())
    cat_att = pd.DataFrame(encoded_data, columns=cat_columns)
    cat_att.index = test_data[a].index

    X_test = pd.concat([test_data[a], cat_att], axis=1)
    y_test = test_data["trips"]

    prediction = fitted.predict(X_test)
    estimate = sum(np.abs(prediction.astype(int) - y_test)) / (102 * 739)
    
    results.append((fitted, estimate))
    print(estimate)

8.474860038738093
10.189087001512378
11.80450529332166
12.39929422377882


In [ ]:
estimate = 0
for i in range(6):
    estimate += results[i][1]
print(estimate / 6)

## Задание № 3

Переобучите итоговые модели на данных до мая 2016 включительно, постройте прогнозы на июнь для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в уже знакомом вам формате: geoID, histEndDay, histEndHour, step, y

In [10]:
train_data = []
control_data = []

for data in result_data:
    train_data.append(pd.concat([data[0], data[1]]))
    control_data.append(data[2])

In [11]:
%%time

file_names =  ["1. first_prediction", "2. second_prediction", "3. third_prediction",
               "4. forth_prediction", "5. fifth_prediction", "6. sixth_prediction"]

results = []
for i in range(len(train_data)):
    estimator = RandomForestRegressor()
    
    data = (train_data[i], control_data[i])
    
    t_data = data[0].replace([np.inf, -np.inf], np.nan).fillna(0.0)
    
    encoded_data = encoder.transform(t_data[cat_attributes].T.to_dict().values())
    cat_att = pd.DataFrame(encoded_data, columns=cat_columns)
    cat_att.index = t_data[a].index

    X_train = pd.concat([t_data[a], cat_att], axis=1)
    y_train = t_data["trips"]

    fitted = estimator.fit(X_train, y_train)
    
    test_data = data[1].replace([np.inf, -np.inf], np.nan).fillna(0.0)
    
    encoded_data = encoder.transform(test_data[cat_attributes].T.to_dict().values())
    cat_att = pd.DataFrame(encoded_data, columns=cat_columns)
    cat_att.index = test_data[a].index

    X_test = pd.concat([test_data[a], cat_att], axis=1)

    prediction = fitted.predict(X_test)
    
    res_shape = test_data["region"].get_values().shape[0]
    
    prediction = pd.DataFrame(np.concatenate((test_data["region"].get_values().reshape(res_shape, 1),
                                              abs(prediction.astype(int)).reshape(res_shape, 1)),
                                             axis=1),
                              columns=["region", "trips"])
    
    prediction.to_csv("new_result/" + file_names[i] + ".csv", index=False)
    
    results.append((fitted, prediction))
    
    print(str(i))

0
1
2
3
4
5
CPU times: user 6h 25min 59s, sys: 19min 44s, total: 6h 45min 43s
Wall time: 6h 50min 28s


In [5]:
start_dt = datetime(2016, 5, 31, 23)
end_dt = datetime(2016, 6, 30, 17)
dates = daterange(start_dt, end_dt)

date_list = list(map(lambda date: np.repeat(date.strftime("_%Y-%m-%d_") + str(date.hour) + "_", 102), dates))
date_list = np.concatenate(date_list)

In [13]:
file_path = np.sort(glob("new_result/*.csv", recursive=True))
solution_data = []
for i in range(len(file_path)):
    path = file_path[i]
    result = pd.read_csv(path)[:len(dates * 102)]
    solution_data.append(result)
    print(result.shape)

(72930, 2)
(72930, 2)
(72930, 2)
(72930, 2)
(72930, 2)
(72930, 2)


In [14]:
solutions = []
for i in range(6):
    res = solution_data[i]
    regions = list(map(lambda reg: reg.split("_")[1], solution_data[i]["region"]))

    ids = []
    for j in range(len(date_list)):
        ids.append(regions[j] + date_list[j])
    
    res["id"] = list(map(lambda _id: _id + str(i+1), ids))
    res["y"] = res.trips
    solutions.append(res[["id", "y"]])

In [15]:
pd.concat(solutions).sort_values("id").to_csv("new_solution.csv", index=False)

## Задание № 4

Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

Текущий результат **11.62815** (Прошлый результат **18.27950**)

https://www.kaggle.com/submissions/6651636/6651636.zip